In [3]:
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob



In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
IMAGE_SIZE = [224, 224]

train_path = '/content/drive/MyDrive/data/train'
valid_path = '/content/drive/MyDrive/data/test'

In [5]:
inception = InceptionV3(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

87910968/87910968 [==============================] - 5s 0us/step


In [10]:
for layer in inception.layers:
    layer.trainable = False

folders = glob('/content/drive/MyDrive/data/train/*')
print(folders)

['/content/drive/MyDrive/data/train/diseased cotton leaf', '/content/drive/MyDrive/data/train/diseased cotton plant', '/content/drive/MyDrive/data/train/fresh cotton leaf', '/content/drive/MyDrive/data/train/fresh cotton plant']


In [12]:
x = Flatten()(inception.output)
prediction = Dense(len(folders), activation='softmax')(x)

In [13]:
model = Model(inputs=inception.input, outputs=prediction)

model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 111, 111, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 111, 111, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 111, 111, 32)         0         ['batch_normalization[0][0

In [14]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [18]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory('/content/drive/MyDrive/data/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('/content/drive/MyDrive/data/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1951 images belonging to 4 classes.
Found 18 images belonging to 4 classes.


In [20]:
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

<ipython-input-20-89113a27acbd>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/5
61/61 [==============================] - 1411s 23s/step - loss: 3.2419 - accuracy: 0.7207 - val_loss: 0.5686 - val_accuracy: 0.8333
Epoch 2/5
61/61 [==============================] - 35s 580ms/step - loss: 0.6181 - accuracy: 0.8929 - val_loss: 0.6169 - val_accuracy: 0.8333
Epoch 3/5
61/61 [==============================] - 37s 602ms/step - loss: 0.6382 - accuracy: 0.8985 - val_loss: 0.1997 - val_accuracy: 0.9444
Epoch 4/5
61/61 [==============================] - 37s 599ms/step - loss: 0.5252 - accuracy: 0.9077 - val_loss: 0.0569 - val_accuracy: 0.9444
Epoch 5/5
61/61 [==============================] - 38s 618ms/step - loss: 0.4928 - accuracy: 0.9206 - val_loss: 0.2050 - val_accuracy: 0.8889
